# Tratamiento del fichero csv de cervezas obtenido de la web *soloartesanas.es*

In [8]:
import numpy as np
import pandas as pd
import yaml

#pd.options.display.max_colwidth = 300

In [2]:
# Importo el fichero "soloartesanas.csv"

In [9]:
df_soloartesanas=pd.read_csv("data/df_soloartesanas.csv", sep=";")
df_soloartesanas.head()

,Unnamed: 0,id,color,name,brand,rate_beer,features,description,image
0,0,1,rubia,"Arriaca Radler, cerveza artesana con limón (lata)",(cervezas artesanas Cervezas Arriaca),NaN,"{'Estilo de cerveza': ' Blonde Ale', 'Graduaci...",\nLa primera cerveza Radler artesana creada en...,https://static2.soloartesanas.es/3437-large_de...
1,1,2,rubia,Alegría & Boris Brew Ipanosuarus Rex,(cervezas artesanas Cervezas Alegría),NaN,{'Estilo de cerveza': ' Imperial / Double IPA'...,"\nUna doble IPA rubia, potente y seca. Elabora...",https://static3.soloartesanas.es/3333-large_de...
2,2,3,rubia,Roy The Bull Mango,(cervezas artesanas Roy The Bull),NaN,"{'Estilo de cerveza': ' Fruit Beer', 'Graduaci...",\nAquí está la primera cerveza de Roy The Bull...,https://static3.soloartesanas.es/3332-large_de...
3,3,4,rubia,Marijuana,(cervezas artesanas Marijuana),NaN,"{'Estilo de cerveza': ' Pale Ale', 'Graduación...",\nLa única cerveza cerveza artesana que podrás...,https://static1.soloartesanas.es/3223-large_de...
4,4,5,rubia,Arriaca Trigo (lata),(cervezas artesanas Cervezas Arriaca),NaN,"{'Estilo de cerveza': ' Weizen - Weissbier', '...",\nUna cerveza de trigo muy afrutada y ligera. ...,https://static2.soloartesanas.es/3449-large_de...


In [10]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   435 non-null    int64 
 1   id           435 non-null    int64 
 2   color        435 non-null    object
 3   name         435 non-null    object
 4   brand        435 non-null    object
 5   rate_beer    157 non-null    object
 6   features     435 non-null    object
 7   description  433 non-null    object
 8   image        435 non-null    object
dtypes: int64(2), object(7)
memory usage: 30.7+ KB


## Limpieza inicial del fichero
Revisión inicial del dataframe para eliminar columnas inncecesarias, agrupar información, unificar tipos y contenidos, etc. de modo que se pueda realizar después un análisis de la información.

Para empezar, gran parte de la información está recogida en el campo "features" que tiene estructura de diccionario y al que habrá que extraer la información separándola en columnas.

In [11]:
# Veo cómo es el campo "features" con un ejemplo
df_soloartesanas.iloc[355, :]['features']

"{'Estilo de cerveza': ' Stout', 'Graduación alcohólica': ' 4,2% Alc./Vol.', 'Amargor (según la Unidad Internacional de Amargor)': ' 40 IBUs', 'Color': 'Negra', 'Grano': 'Avena', 'Fermentación': 'Ale', 'European Brewing Convention (EBC)': '110', 'Maltas': 'Maris Otter Floor Malt, Chocolate, Roasted Barley, Crystal y Avena', 'Lúpulos': 'Cascade', 'Población': 'Liérganes', 'Provincia': 'Cantabria', 'Comunidad Autónoma': 'Cantabria', 'País': 'España'}"

In [12]:
# para ver las características del registro más largo

df_soloartesanas["num_caract"]= df_soloartesanas['features'].apply(len)

features_mas_largo=df_soloartesanas[df_soloartesanas["num_caract"]==df_soloartesanas["num_caract"].max()]['features']
df_soloartesanas.drop(columns="num_caract",inplace=True)

features_mas_largo.values

array(["{'Estilo de cerveza': ' Amber Ale', 'Graduación alcohólica': ' 5.3% Alc./Vol.', 'Amargor (según la Unidad Internacional de Amargor)': ' 30 IBUs', 'Color': 'Tostada', 'Grano': 'Cebada', 'Fermentación': 'Ale', 'European Brewing Convention (EBC)': '40', 'Maltas': 'Pale Ale, Crystal, Victoria y CaraAroma', 'Lúpulos': 'Kent Golding y Fuggles', 'Población': 'Oviedo', 'Provincia': 'Asturias', 'Comunidad Autónoma': 'Asturias', 'País': 'España', 'Temperatura de consumo': '10-13ºC', 'Vaso recomendado': 'Vaso ancho'}"],
      dtype=object)

In [13]:
# Hay que transformar la columna de features, que es un diccionario, en varias columnas 
df_soloartesanas['features']

0      {'Estilo de cerveza': ' Blonde Ale', 'Graduaci...
1      {'Estilo de cerveza': ' Imperial / Double IPA'...
2      {'Estilo de cerveza': ' Fruit Beer', 'Graduaci...
3      {'Estilo de cerveza': ' Pale Ale', 'Graduación...
4      {'Estilo de cerveza': ' Weizen - Weissbier', '...
                             ...                        
430    {'Estilo de cerveza': ' English Pale Ale', 'Gr...
431    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
432    {'Estilo de cerveza': ' Irish Red Ale', 'Gradu...
433    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
434    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
Name: features, Length: 435, dtype: object

In [14]:
# aplicar yaml a columna features y convertirla en diccionario aplicando map
# Para un registro sería: mi_dict2 = yaml.load(df_soloartesanas.features[0])

df_soloartesanas['features'] = df_soloartesanas.features.map(lambda x: yaml.load(x))

<ipython-input-14-f3f490e69a13>:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  df_soloartesanas['features'] = df_soloartesanas.features.map(lambda x: yaml.load(x))


In [15]:
df_soloartesanas['features']

0      {'Estilo de cerveza': ' Blonde Ale', 'Graduaci...
1      {'Estilo de cerveza': ' Imperial / Double IPA'...
2      {'Estilo de cerveza': ' Fruit Beer', 'Graduaci...
3      {'Estilo de cerveza': ' Pale Ale', 'Graduación...
4      {'Estilo de cerveza': ' Weizen - Weissbier', '...
                             ...                        
430    {'Estilo de cerveza': ' English Pale Ale', 'Gr...
431    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
432    {'Estilo de cerveza': ' Irish Red Ale', 'Gradu...
433    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
434    {'Estilo de cerveza': ' Red Ale', 'Graduación ...
Name: features, Length: 435, dtype: object

In [16]:
# veo un ejemplo
print(df_soloartesanas.iloc[355, :])

Unnamed: 0                                                   355
id                                                           356
color                                                      negra
name                                     Dougall's Session Stout
brand                             (cervezas artesanas Dougall's)
rate_beer                                               92 / 100
features       {'Estilo de cerveza': ' Stout', 'Graduación al...
description    \nDougalls Session Stout es una excelente cerv...
image          https://static3.soloartesanas.es/2095-large_de...
Name: 355, dtype: object


In [17]:
# Separo en columnas cada clave/valor del dicc de features

df_soloartesanas = pd.concat([df_soloartesanas.drop(['features'], axis=1), df_soloartesanas['features'].apply(pd.Series)], axis=1)

In [18]:
df_soloartesanas.head(2)

,Unnamed: 0,id,color,name,brand,rate_beer,description,image,Estilo de cerveza,Graduación alcohólica,...,Comunidad Autónoma,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Vaso recomendado,Grano,Sin gluten,Ecológica,País
0,0,1,rubia,"Arriaca Radler, cerveza artesana con limón (lata)",(cervezas artesanas Cervezas Arriaca),NaN,\nLa primera cerveza Radler artesana creada en...,https://static2.soloartesanas.es/3437-large_de...,Blonde Ale,"2,7% Alc./Vol.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,rubia,Alegría & Boris Brew Ipanosuarus Rex,(cervezas artesanas Cervezas Alegría),NaN,"\nUna doble IPA rubia, potente y seca. Elabora...",https://static3.soloartesanas.es/3333-large_de...,Imperial / Double IPA,"8,2% Alc./Vol.",...,Valencia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Compruebo las nuevas columnas de features, tus tipos y misings
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 25 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   Unnamed: 0                                          435 non-null    int64 
 1   id                                                  435 non-null    int64 
 2   color                                               435 non-null    object
 3   name                                                435 non-null    object
 4   brand                                               435 non-null    object
 5   rate_beer                                           157 non-null    object
 6   description                                         433 non-null    object
 7   image                                               435 non-null    object
 8   Estilo de cerveza                                   419 non-null    object
 9   Graduación

In [14]:
# Algunas de estas nuevas columnas las podré borrar si no aportan información y no están en los otros ficheros
# Otras las puedo unir en una única columna, como el origen
# Algunas características que no estén completas, si aparecen en otros ficheros, las dejaré con sus NAN
# En la app podrán seleccionarse los campos a desplegar en función de si hay dato

In [15]:
'''
RangeIndex: 435 entries, 0 to 434
Data columns (total 25 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   Unnamed: 0                                          435 non-null    int64 -- Borrar
 1   id                                                  435 non-null    int64 
 2   color                                               435 non-null    object
 3   name                                                435 non-null    object
 4   brand                                               435 non-null    object
 5   rate_beer                                           157 non-null    object -- Borrar
 6   description                                         433 non-null    object -- completar
 7   image                                               435 non-null    object
 8   Estilo de cerveza                                   419 non-null    object
 9   Graduación alcohólica                               415 non-null    object
 10  Color                                               418 non-null    object -- Borrar (duplicada)
 11  Fermentación                                        399 non-null    object
 12  Amargor (según la Unidad Internacional de Amargor)  275 non-null    object
 13  Población                                           409 non-null    object -- Unificar en "Origen"
 14  Provincia                                           426 non-null    object -- Unificar en "Origen"
 15  Comunidad Autónoma                                  424 non-null    object -- Unificar en "Origen"
 16  Lúpulos                                             99 non-null     object
 17  European Brewing Convention (EBC)                   101 non-null    object
 18  Maltas                                              87 non-null     object
 19  Temperatura de consumo                              137 non-null    object
 20  Vaso recomendado                                    52 non-null     object -- Borrar/añadir a descr
 21  Grano                                               310 non-null    object
 22  Sin gluten                                          2 non-null      object -- Borrar
 23  Ecológica                                           3 non-null      object -- Borrar
 24  País                                                45 non-null     object -- Unificar en "Origen"
dtypes: int64(2), object(23)

'''
print(" ")

In [20]:
# compruebo si las cervezas sin gluten y las ecológicas tienen esa info en su descripción (se podrían borrar)
sin=df_soloartesanas[df_soloartesanas["Sin gluten"]=='No']["description"]
list(sin)

['\nUna cerveza artesana para disfrutar del buen tiempo y del calor que recorre nuestra geografía durante los meses cálidos. Los chicos de La Virgen elaboran La Virgen Veraniega para época\xa0estival con fecha límite de producción, Septiembre. Y lo hacen para ofrecerte una cerveza genuina y adaptada a las exigencias meteorológicas del calor.\nEs una cerveza de estilo Pale Ale que usa levadura de alta fermentación California Lager. Todo acompañado de una incipiente presencia de amargor con un gran equilibrio a malta, hace de ella una cerveza artesana muy refrescante.\nDisfruta y refréscate este verano\n']

In [17]:
sin=df_soloartesanas[df_soloartesanas["Ecológica"]=='Sí']["description"]
list(sin)

['\nLlega una nueva evolución de Armando... que ahora se convierte en "The Tree Man", una cerveza orgánica y ecológica (como toda artesana, sin aditivos ni conservantes ni colorantes ni acelerantes). Una cerveza 100% natural.\nDisfruta de esta cerveza de estilo Pale Ale de corte inglés, fresca y con aromas y matices a frutas tropicales.\n',
 '\nBeauy es una cerveza completa. Es una gran cerveza: es ecológica, es artesana y uno de sus principales componentes es el Aloe Vera.\nLas base en una Pale Ale de estilo americano, sobre la que se elabora esta cerveza vegetal suave, equilibrada y ligera. Gran presencia dorada en copa, de la que saldran los primeros aromas de miel y caramelo, y al instante llegará el herbal del aloe. En boca resulta poco amarga y muy fresca, creando una cerveza muy fácil de beber.\nDisfruta de la buena cerveza\n']

In [21]:
# borro las columnas que no aportan valor por estar repetidas o tener muy pocos datos: 
# "Unnamed: 0","Vaso recomendado","Sin gluten","Ecológica" y "Color"

df_soloartesanas.drop(columns=["Unnamed: 0","rate_beer","Color","Vaso recomendado","Sin gluten","Ecológica"], inplace=True)

In [22]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 19 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   id                                                  435 non-null    int64 
 1   color                                               435 non-null    object
 2   name                                                435 non-null    object
 3   brand                                               435 non-null    object
 4   description                                         433 non-null    object
 5   image                                               435 non-null    object
 6   Estilo de cerveza                                   419 non-null    object
 7   Graduación alcohólica                               415 non-null    object
 8   Fermentación                                        399 non-null    object
 9   Amargor (s

In [23]:
# veo qué cervezas están sin descripción
df_soloartesanas[df_soloartesanas["description"].isna()]

,id,color,name,brand,description,image,Estilo de cerveza,Graduación alcohólica,Fermentación,Amargor (según la Unidad Internacional de Amargor),Población,Provincia,Comunidad Autónoma,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Grano,País
171,172,rubia,Catalina,(cervezas artesanas Torquemada),NaN,https://static2.soloartesanas.es/886-large_def...,Pilsner,"4,8% Alc./Vol.",Ale,NaN,Torquemada,Palencia,Castilla y León,NaN,NaN,NaN,2-4ºC,Cebada,NaN
223,224,rubia,Illusió,(cervezas artesanas Ginesart),NaN,https://static2.soloartesanas.es/2664-large_de...,Iber Ale,"5,0% Alc./Vol.",Ale,NaN,Ginesart,Tarragona,Cataluña,Cascade y Saaz. Salvo,NaN,"Pale ale, Pilsen y de trigo",8-12 ºC,Trigo,NaN


In [24]:
# Añado una descripción de las cervezas que faltan y que encuentro en otras webs
df_soloartesanas.loc[171,"description"]="Se trata de una cerveza rubia tipo Pils, ligera y refrescante a base de malta Pilsner y lúpulos alemanes y checos. Catalina debe su nombre al nacimiento, el 14 de enero de 1507, de la Infanta Catalina de Austria (reina de Portugal) en la localidad Torquemada. Hija de Juana de Castilla y Felipe de Habsburgo. La cerveza Catalina es una cerveza rubia brillante, ligeramente amarga, con olores a cereal y una espuma equilibrada. El contenido alcohólico es de 4,8% Vol. lo que hace que esta cerveza sea la más ligera de toda la gama. Temperatura de consumo recomendada: 2º-4º"
df_soloartesanas.loc[223,"description"]="Estilo cervecero: Iber Ale. Cerveza rubia de alta fermentación adaptada al estilo de nuestro país. Nota de cata: Toda la intensidad del lúpulo en combinación con la malta, para hacer pasar la sed. Ingredientes: Malta Pale ale, Pilsen y de trigo. de los lúpulos: Cascade y Saaz. Levadura. Temperatura de consumo: entre 8 y 12 º C Graduación alcohólica: 5% Alc. Vol."

In [25]:
df_soloartesanas.iloc[223]["description"]

'Estilo cervecero: Iber Ale. Cerveza rubia de alta fermentación adaptada al estilo de nuestro país. Nota de cata: Toda la intensidad del lúpulo en combinación con la malta, para hacer pasar la sed. Ingredientes: Malta Pale ale, Pilsen y de trigo. de los lúpulos: Cascade y Saaz. Levadura. Temperatura de consumo: entre 8 y 12 º C Graduación alcohólica: 5% Alc. Vol.'

### Unifico la información sobre el origen de fabricación
Por la propia página web sabemos que todas las cervezas son de origen español, pero revisamos los datos para ver si falta información y si hay uniformidad en la nomenclaruta.

In [26]:
# compruebo que efectivamente solo hay cervezas españolas
print("Sin país: ", df_soloartesanas["País"].isna().sum())
df_soloartesanas.groupby("País")["id"].nunique()

Sin país:  390


País
ESpaña     1
España    44
Name: id, dtype: int64

Hay demasiadas cervezas sin identifiador de país. Puedo forzar la columna y poner en todas "España", pero antes voy a comprobar los otros campos sobre población, provincia y comunidad autónoma, para asegurarme de que no hay cervezas de otros.

Del total de 435 tenemos datos sobre:
- Población:                                           409 
- Provincia:                                           426 
- Comunidad Autónoma:                                 424 



In [27]:
# Compruebo las provincias que aparecen
df_soloartesanas["Provincia"].unique()

array([nan, 'Valencia', 'Pontevedra', 'Ávila', 'Guadalajara', 'Barcelona',
       'Segovia', 'Alicante', 'Burgos', 'La Rioja', 'Vizcaya', 'Madrid',
       'Córdoba', 'Málaga', 'Teruel', 'Valladolid', 'Badajoz',
       'Castellón', 'Cuenca', 'Tarragona', 'Girona', 'Álava', 'Albacete',
       'Soria', 'Lugo', 'Navarra', 'León', 'Palencia', 'Murcia', 'Huelva',
       'Asturias', 'Toledo', 'Cantabria', 'Sevilla', 'Ciudad Real',
       'Orense', 'La Coruña', 'Huesca', 'Lérida', 'Cádiz', 'Granada',
       'Guipúzcoa', 'Zaragoza', 'Zamora', 'Salamanca', 'Cáceres'],
      dtype=object)

In [28]:
# Estas son las CC.AA que aparecen
df_soloartesanas["Comunidad Autónoma"].unique()

array([nan, 'Valencia', 'Galicia', 'Castilla y León',
       'Castilla-La Mancha', 'Cataluña', 'La Rioja', 'País Vasco',
       'Madrid', 'Andalucía', 'Aragón', 'Extremadura', 'Navarra',
       'Murcia', 'Asturias', 'Canarias', 'Cantabria'], dtype=object)

In [29]:
# ¿Que CCAA faltan de las que sí sabemos la provincia?
df_soloartesanas[df_soloartesanas["Comunidad Autónoma"].isna()]["Provincia"]


0         NaN
29        NaN
30        NaN
41        NaN
63     Girona
69        NaN
230       NaN
351    Girona
352    Girona
372       NaN
396       NaN
Name: Provincia, dtype: object

In [31]:
# como son todas de Girona, actualizamos la CCAA con Cataluña
mask=(df_soloartesanas["Comunidad Autónoma"].isna()) & (df_soloartesanas["Provincia"]=="Girona")
df_soloartesanas.loc[mask,"Comunidad Autónoma"]="Cataluña"

In [33]:
# comprobamos que se han actualizado bien
df_soloartesanas[df_soloartesanas["Comunidad Autónoma"].isna()]["Provincia"]

0      NaN
29     NaN
30     NaN
41     NaN
69     NaN
230    NaN
372    NaN
396    NaN
Name: Provincia, dtype: object

In [34]:
# A la inversa, en las provincias faltan ¿cuál es la CC.AA.?
df_soloartesanas[df_soloartesanas["Provincia"].isna()]["Comunidad Autónoma"]

0           NaN
29          NaN
30          NaN
41          NaN
69          NaN
230         NaN
257    Cataluña
372         NaN
396         NaN
Name: Comunidad Autónoma, dtype: object

Comprobamos que las 8 provincias que faltan no tienen información sobre CC.AA, salvo 1 que es de Cataluña.
El siguiente paso será averiguar qué información tenemos y falta respecto a la población de origen, para ver si se pueden completar las otras columnas.

In [35]:
# Por un lado, en las provincias que faltan ¿cuál es población?
df_soloartesanas[df_soloartesanas["Provincia"].isna()]["Población"]

0             NaN
29            NaN
30            NaN
41            NaN
69            NaN
230           NaN
257    Granollers
372           NaN
396           NaN
Name: Población, dtype: object

In [36]:
# Lo mismo para las CC.AA.
df_soloartesanas[df_soloartesanas["Comunidad Autónoma"].isna()]["Población"]

0      NaN
29     NaN
30     NaN
41     NaN
69     NaN
230    NaN
372    NaN
396    NaN
Name: Población, dtype: object

Con la información de la columna "Población" solamente podemos completar los datos del registro 257 (Granollers), que corresponden a la provincia de Barcelona en Cataluña.

In [37]:
# Actualizo el registro 257 (Granollers)
df_soloartesanas.loc[257,"Provincia"]="Barcelona"

In [38]:
print("Cervezas con población de origen",df_soloartesanas["Población"].count())
print("Cervezas con provincia",df_soloartesanas["Provincia"].count())
print("Cervezas con CC.AA",df_soloartesanas["Comunidad Autónoma"].count())


Cervezas con población de origen 409
Cervezas con provincia 427
Cervezas con CC.AA 427


Esto significa que de las 435 cervezas recogidas 409 tienen toda la información, 8 no tienen ningún dato sobre procedencia y 18 tienen información regional.
Dada esta circunstancia, se decide:
1. Eliminar los 8 registros con origen totalmente desconocido.
2. Conservar los 18 incompletos, indicando que la población de origen está "sin especificar". 
3. Unificar como país España

Con ello, conseguiríamos un dataset con 427 registros documentados.

In [39]:
# Elimino los 8 registros sin ninguna información sobre su origien
mask=df_soloartesanas[df_soloartesanas["Comunidad Autónoma"].isna()]
df_soloartesanas=df_soloartesanas.drop(mask.index)


In [42]:
# Asigno "sin especificar" a las poblaciones que faltan pero sí conocemos su provincia y CC.AA
df_soloartesanas["Población"]=df_soloartesanas["Población"].fillna("sin especificar")

In [46]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 427 entries, 1 to 434
Data columns (total 19 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   id                                                  427 non-null    int64 
 1   color                                               427 non-null    object
 2   name                                                427 non-null    object
 3   brand                                               427 non-null    object
 4   description                                         427 non-null    object
 5   image                                               427 non-null    object
 6   Estilo de cerveza                                   416 non-null    object
 7   Graduación alcohólica                               412 non-null    object
 8   Fermentación                                        396 non-null    object
 9   Amargor (s

In [37]:
df_soloartesanas.columns

Index(['id', 'color', 'name', 'brand', 'description', 'image',
       'Estilo de cerveza', 'Graduación alcohólica', 'Fermentación',
       'Amargor (según la Unidad Internacional de Amargor)', 'Población',
       'Provincia', 'Comunidad Autónoma', 'Lúpulos',
       'European Brewing Convention (EBC)', 'Maltas', 'Temperatura de consumo',
       'Grano', 'País'],
      dtype='object')

In [47]:
# Unifico el campo "País" que tiene Nan y distintas grafías
df_soloartesanas["País"]="España"

# Muevo las columnas para que la información sobre la procedencia esté ordenada y al final del dataset
df_soloartesanas=df_soloartesanas[['id', 'color', 'name', 'brand', 'description', 'image',
       'Estilo de cerveza', 'Graduación alcohólica', 'Fermentación',
       'Amargor (según la Unidad Internacional de Amargor)', 'Lúpulos',
       'European Brewing Convention (EBC)', 'Maltas', 'Temperatura de consumo',
       'Grano', 'Población',
       'Provincia', 'Comunidad Autónoma', 'País']]


## Normalización de datos específicos sobre cervezas
A continuación se procede a revisar las columnas con información específica sobre las características de las cervezas, con el objetivo de que los datos sean uniformes y los tipos se correspondan con ellos. También se cambiarán los nombres de algunas columnas que resultan demasiado largos para trabajar.

In [48]:
df_soloartesanas.columns

Index(['id', 'color', 'name', 'brand', 'description', 'image',
       'Estilo de cerveza', 'Graduación alcohólica', 'Fermentación',
       'Amargor (según la Unidad Internacional de Amargor)', 'Lúpulos',
       'European Brewing Convention (EBC)', 'Maltas', 'Temperatura de consumo',
       'Grano', 'Población', 'Provincia', 'Comunidad Autónoma', 'País'],
      dtype='object')

In [49]:
# cambio nombres de columnas demasiado largos
df_soloartesanas.rename(columns={'Estilo de cerveza':"Estilo",'Graduación alcohólica':"Graduación",'Amargor (según la Unidad Internacional de Amargor)':"IBU", },inplace=True)

In [50]:
# Modifico la columna de porcentaje de alcohol (ahora es texto) para dejar graduación como float

# Valor ficticio(999) para evitar errores con los NAN
df_soloartesanas['Graduación']=df_soloartesanas['Graduación'].fillna("999")

# sustituir comas por puntos
df_soloartesanas['Graduación']=df_soloartesanas['Graduación'].apply(lambda x: x.replace(',','.'))

# extraer el dato numérico (nums antes y después de .)
df_soloartesanas['Graduación'] = df_soloartesanas["Graduación"].str.extract(r"(\d+(.\d+){0,1})", expand=True)[0]

# transformar en float
df_soloartesanas['Graduación']=df_soloartesanas['Graduación'].astype('float64')

In [51]:
df_soloartesanas['Graduación'].unique()

array([  8.2 ,   6.  ,   5.7 ,   4.7 ,   6.6 ,   5.4 ,   4.5 ,   3.1 ,
         5.8 ,   6.7 ,   3.6 ,   5.1 ,   5.2 ,   5.5 ,   6.8 ,   5.6 ,
         6.2 ,   5.  ,   6.5 ,   6.4 ,   8.5 ,   5.3 ,   7.9 ,   4.3 ,
         8.7 ,   7.5 ,   5.9 ,   4.4 ,   4.  ,   7.2 ,   9.  ,   7.6 ,
         8.  ,   7.1 ,   4.9 ,   9.3 , 999.  ,   4.2 ,   7.  ,   4.8 ,
         4.6 ,   6.1 ,  10.1 ,   2.5 ,   6.3 ,   3.5 ,   6.9 ,   9.1 ,
         2.6 ,   7.3 ,  14.  ,   9.5 ,   7.8 ,  10.4 ,  10.5 ,  11.  ,
        11.2 ,   5.85])

In [52]:
df_soloartesanas["IBU"].unique()

array([' 60 IBUs', ' 38 IBUs', nan, ' 50 IBUs', ' 20 IBUs', ' 28 IBUs',
       ' 12 IBUs', ' 10 IBUs', ' 35 IBUs', ' 26 IBUs', ' 30 IBUs',
       ' 18 IBUs', ' 70 IBUs', ' 21 IBUs', ' 40 IBUs', ' 24 IBUs',
       ' 13 IBUs', ' 44 IBUs', ' 23 IBUs', ' 31 IBUs', ' 45 IBUs',
       ' 85 IBUs', ' 65 IBUs', ' 52 IBUs', ' 100 IBUs', ' 25 IBUs',
       ' 48 IBUs', ' 22 IBUs', ' 42 IBUs', ' 14 IBUs', ' 47 IBUs',
       ' 75 IBUs', ' 27 IBUs', ' 66 IBUs', ' 49 IBUs', ' 130 IBUs',
       ' 13,8 IBUs', ' 110 IBUs', ' 37 IBUs', ' 46 IBUs', ' 17,9 IBUs',
       ' 55 IBUs', ' 32 IBUs', ' 15 IBUs', ' 39 IBUs', ' 54 IBUs',
       ' 8 IBUs', ' 19 IBUs', ' 11 IBUs', ' 80 IBUs', ' 56 IBUs',
       ' 29 IBUs', ' 120 IBUs', ' 59 IBUs', ' 23,5 IBUs', ' 36 IBUs',
       ' ND IBUs', ' 17 IBUs', ' 34 IBUs', ' 22,4 IBUs', ' 41 IBUs',
       ' 43 IBUs', ' 57,7 IBUs', ' 84 IBUs', ' 62 IBUs', ' 33 IBUs',
       ' 90 IBUs', ' 15.2 IBUs', ' 72 IBUs', ' 69 IBUs', ' 64 IBUs',
       ' 92 IBUs', ' 9,9 IBUs', ' 150 IBUs

In [53]:
# limpio IBUS para dejar solo los números

# Valor ficticio(999) para evitar errores con los NAN
df_soloartesanas['IBU']=df_soloartesanas['IBU'].fillna("999")

# sustituir comas por puntos
df_soloartesanas['IBU']=df_soloartesanas['IBU'].apply(lambda x: x.replace(',','.'))

# extraer el dato numérico (nums antes y después de .)
df_soloartesanas['IBU']=df_soloartesanas['IBU'].str.extract(r"(\d+(.\d+){0,1})", expand=True)[0]

# transformar en float
df_soloartesanas['IBU']=df_soloartesanas['IBU'].fillna("999")
df_soloartesanas['IBU']=df_soloartesanas['IBU'].astype('float64')
df_soloartesanas['IBU'].unique()

array([ 60. ,  38. , 999. ,  50. ,  20. ,  28. ,  12. ,  10. ,  35. ,
        26. ,  30. ,  18. ,  70. ,  21. ,  40. ,  24. ,  13. ,  44. ,
        23. ,  31. ,  45. ,  85. ,  65. ,  52. , 100. ,  25. ,  48. ,
        22. ,  42. ,  14. ,  47. ,  75. ,  27. ,  66. ,  49. , 130. ,
        13.8, 110. ,  37. ,  46. ,  17.9,  55. ,  32. ,  15. ,  39. ,
        54. ,   8. ,  19. ,  11. ,  80. ,  56. ,  29. , 120. ,  59. ,
        23.5,  36. ,  17. ,  34. ,  22.4,  41. ,  43. ,  57.7,  84. ,
        62. ,  33. ,  90. ,  15.2,  72. ,  69. ,  64. ,  92. ,   9.9,
       150. ])

In [45]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 427 entries, 1 to 434
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 427 non-null    int64  
 1   color                              427 non-null    object 
 2   name                               427 non-null    object 
 3   brand                              427 non-null    object 
 4   description                        427 non-null    object 
 5   image                              427 non-null    object 
 6   Estilo                             416 non-null    object 
 7   Graduación                         427 non-null    float64
 8   Fermentación                       396 non-null    object 
 9   IBU                                427 non-null    float64
 10  Lúpulos                            98 non-null     object 
 11  European Brewing Convention (EBC)  101 non-null    object 

In [46]:
df_soloartesanas['brand'].value_counts()

(cervezas artesanas Birra & Blues)       13
(cervezas artesanas Cervezas Arriaca)    12
(cervezas artesanas Althaia Artesana)    10
(cervezas artesanas Dawat)                9
(cervezas artesanas Sagra)                9
                                         ..
(cervezas artesanas Senador Volstead)     1
(cervezas artesanas Cartujana)            1
(cervezas artesanas Del Pueblo)           1
(cervezas artesanas Mateo & Bernabé)      1
(cervezas artesanas Bronhër)              1
Name: brand, Length: 143, dtype: int64

In [54]:
# eliminar "cervezas artesanas " y los paréntesis en el campo "brand"
df_soloartesanas['brand']=df_soloartesanas['brand'].apply(lambda x: x.replace('(cervezas artesanas ',''))
df_soloartesanas['brand']=df_soloartesanas['brand'].apply(lambda x: x.replace(')',''))

In [48]:
# ejemplo de una cerveza
df_soloartesanas.iloc[223]

id                                                                                 229
color                                                                            rubia
name                                                                Gredos Doble Malta
brand                                                                   Cerveza Gredos
description                          \nEsta es la cerveza artesana doble malta de C...
image                                https://static2.soloartesanas.es/392-large_def...
Estilo                                                              American Amber Ale
Graduación                                                                         6.9
Fermentación                                                                       Ale
IBU                                                                                999
Lúpulos                                                                            NaN
European Brewing Convention (EBC)          

In [49]:
df_soloartesanas.columns

Index(['id', 'color', 'name', 'brand', 'description', 'image', 'Estilo',
       'Graduación', 'Fermentación', 'IBU', 'Lúpulos',
       'European Brewing Convention (EBC)', 'Maltas', 'Temperatura de consumo',
       'Grano', 'Población', 'Provincia', 'Comunidad Autónoma', 'País'],
      dtype='object')

In [50]:
df_soloartesanas["color"].unique()

array(['rubia', 'tostada', 'negra', 'blanca', 'roja'], dtype=object)

In [51]:
df_soloartesanas["name"].unique()

array(['Alegría & Boris Brew Ipanosuarus Rex', 'Roy The Bull Mango',
       'Marijuana', 'Arriaca Trigo (lata)', 'Bertus White IPA',
       'Bertus Summer Ale', 'Bertus Carpe Diem', '90 Varas La Mala',
       '90 Varas Zerezo', 'Althaia Rabosa', 'Mica Raíz',
       'Arriaca Session IPA (lata)', 'OOB Hoppy Blonde',
       'Familia Serra Maula', 'Familia Serra Butoni',
       'Familia Serra Mussa', 'Sargs IPA', 'Birra & Blues Mago de Oz',
       'Laugar Basurde', 'Laugar EPA!', 'Birra & Blues Smoke my Beer',
       'Birra & Blues Amber', 'Birra & Blues IPA Blues',
       'Birra & Blues Tripel', 'Birra & Blues La Rubia',
       'Birra & Blues Doble Malta', 'Madriz Chueca', 'Alma Turdetana',
       'La Virgen IPA', 'La Quarantamaula American Pale Ale',
       'La Quarantamaula American Golden Ale', '3Monos Vito Hopleone',
       '3Monos Dirty Harry', '3Monos Monkey Business',
       '3Monos Golden Ape', 'Skunk Diaple',
       'The Flying Inn Liquid Nemesis',
       'The Flying Inn Telepath

In [52]:
df_soloartesanas["brand"].unique()

array(['Cervezas Alegría', 'Roy The Bull', 'Marijuana',
       'Cervezas Arriaca', 'Bertus', '90 Varas', 'Althaia Artesana',
       'Cervezas Mica', 'OOB', 'Familia Serra', 'Sargs', 'Birra & Blues',
       'Laugar', 'Madriz', 'Alma Turdetana', 'Cerveza La Virgen',
       'La Quarantamaula', '3Monos', 'Ordio Minero', 'The Flying Inn',
       'Fundación Badalona Capaç', 'Cervecera Libre', 'Cerveza Ballut',
       'Castelló Beer Factory', 'Dawat', "L'anjub", 'Cervesa Marina',
       'Quana', 'Panda Beer', 'Garagart', 'Ausesken', 'Del Pueblo',
       'Bonvivant', 'La Calavera', 'Arévaka', 'Chacueca', 'Tyris',
       'Cervezas Quijota', 'Cervezas Abadía', 'Morlaco Beer', 'Kadabra',
       'Espina de Ferro', 'Pato Nero', 'Cañonita', 'Cerveza Tartessos',
       'As Cervesa Artesana', 'Cerveza Odiel', 'Cerveses La Pirata',
       'Naparbier', 'Mateo & Bernabé', 'Nurse', 'Cervezas Speranto',
       'Goose', 'Nómada Brewing', 'Smach', 'Cervezas Libre', "Dougall's",
       'Drunken Bros', 'Beauty

### Datos sobre el estilo de las cervezas
Uno de los datos más importantes en la valoración de las cervezas es su estilo. Encontramos que este campo tiene algunos registros sin especificar, por lo que buscamos la manera de completar la información basándonos en otros datos o, de manera puntual, buscando el estilo para unos casos concretos.


In [61]:
df_soloartesanas["Estilo"].value_counts()

 Pale Ale                             34
 India Pale Ale - IPA                 19
 Brown Ale                            17
 Blonde Ale                           14
 Amber Ale                            11
                                      ..
 Smoke Flavored and Wood Aged Beer     1
 English Winter Ale                    1
 Weizen Weissbier                      1
 Red IPA                               1
 Smoked Beer                           1
Name: Estilo, Length: 152, dtype: int64

In [62]:
df_soloartesanas["Estilo"].nunique()

152

In [63]:
df_soloartesanas["Estilo"].unique()

array([' Imperial / Double IPA', ' Fruit Beer', ' Pale Ale',
       ' Weizen - Weissbier', ' White IPA', ' Kölsch',
       ' English Pale Ale', ' Lager', ' Pumpkin Ale',
       ' India Pale Ale - IPA', ' Session IPA', ' Blonde Ale',
       ' Golden Ale', ' Witbier (Belgian White)', ' Amber Ale',
       ' American Pale Ale - APA', ' Smoked Beer', ' American Amber Ale',
       ' English IPA', ' Tripel', ' Belgian Blonde Ale',
       ' Golden Braggot', ' American Golden Ale', ' Double Red IPA',
       ' American IPA', ' Saison', ' Extra Pale Ale', ' Braggot',
       ' German Ale', ' Rauchbier', ' Belgian Dubbel', ' Belgian IPA',
       ' Cream Ale', ' Strong Ale', nan, ' Double IPA',
       ' California Common / Steam Beer', ' Farmhouse Ale',
       ' American Wheat', ' Spice / Herb / Vegetable',
       ' Pilsner - Pilsen', ' Summer Ale', ' India Pale Ale',
       ' Pilsner Ale', ' Belgian Ale', ' Biére de Garde',
       ' Wheat Ale Ecológica', ' Marzenbier', ' IPA con algarroba',
       

In [64]:
# ¿cómo son las que no tienen estilo?
df_estilo=df_soloartesanas[df_soloartesanas["Estilo"].isna()]

df_estilo=df_estilo.drop(columns=["id", "name","Graduación", "IBU","Lúpulos",
                                  "European Brewing Convention (EBC)","Maltas",
                                  'image',"brand","description",
                                 'Población', 'Provincia',
                                 'Comunidad Autónoma',
                                 'Temperatura de consumo', 'País'])
df_estilo

,color,Estilo,Fermentación,Grano
85,rubia,NaN,Ale,Cebada
172,rubia,NaN,Ale,Trigo
173,rubia,NaN,Ale,Cebada
184,rubia,NaN,Ale,Trigo
185,rubia,NaN,Ale,Cebada
190,rubia,NaN,Ale,Cebada
279,tostada,NaN,Ale,Trigo
286,tostada,NaN,Ale,Cebada
386,negra,NaN,Ale,Cebada
389,negra,NaN,Ale,Cebada


Observamos que para las 11 cervezas que no tienen clasificación de estilo podemos obtener el dato conociendo la fermentación, el color y el grano. 
Por tanto, prodecemos a obtener el estilo que más se repite en cada una de esas combinaciones y se lo añadimos a la cerveza. 

In [65]:
# Eliminamos las combinaciones que se repiten
df_estilo.drop_duplicates(inplace=True)

#inicializamos el índice
df_estilo.reset_index(drop=True,inplace=True)
df_estilo

,color,Estilo,Fermentación,Grano
0,rubia,NaN,Ale,Cebada
1,rubia,NaN,Ale,Trigo
2,tostada,NaN,Ale,Trigo
3,tostada,NaN,Ale,Cebada
4,negra,NaN,Ale,Cebada
5,roja,NaN,Ale,Cebada


In [66]:
# Buscamos el estilo que es la moda para cada combinación (ej.rubia & Ale & Cebada) y lo asignamos a cada cerveza sin estilo

combinaciones=range(df_estilo.columns.size+1)

for comb in combinaciones:
    
    mask=(df_soloartesanas["color"]==df_estilo.loc[comb,"color"])&(df_soloartesanas["Fermentación"]==df_estilo.loc[comb,"Fermentación"])&(df_soloartesanas["Grano"]==df_estilo.loc[comb,"Grano"])
    mk_sin_estilo=mask & df_soloartesanas["Estilo"].isna()

    nuevo_estilo=df_soloartesanas[mask]["Estilo"].mode()
    
    df_soloartesanas.loc[mk_sin_estilo,"Estilo"]=nuevo_estilo[0]

# comprobamos
df_soloartesanas[df_soloartesanas["Estilo"].isna()]


,id,color,name,brand,description,image,Estilo,Graduación,Fermentación,IBU,Lúpulos,European Brewing Convention (EBC),Maltas,Temperatura de consumo,Grano,Población,Provincia,Comunidad Autónoma,País
428,429,roja,Arriaca IPA,Cervezas Arriaca,\nArrica IPA es una de las cervezas artesanas ...,https://static2.soloartesanas.es/3441-large_de...,NaN,999.0,Ale,999.0,NaN,NaN,NaN,NaN,Cebada,sin especificar,Guadalajara,Castilla-La Mancha,España


In [67]:
# queda 1 cerveza sin estilo. Vemos su descripción para buscar más información
df_soloartesanas.loc[428,"description"]

'\nArrica IPA\xa0es una de las\xa0cervezas artesanas que elabora cervezas arriaca desde Yunquera de Henares, Guadalajara.\xa0Es una cerveza artesana IPA, Ale, con marcado carácter a lúpulo. \nDestaca su color\xa0color anaranjado fuerte tirando a rojizo, con una corona de espuma cremosa y de buena retención. Aromas cítricos, sobre todo a pomelo y mandarina, notas herbáceas, florales y matices afrutados.\xa0En boca presenta un amargor medio que se disipa rápidamente dejando paso al cítrico, al pino y al caramelo.\n¡Salud!\n'

In [68]:
# Por la descripción encajaría en una Red IPA. Actualizamos el dato
df_soloartesanas.loc[428,"Estilo"]=' Red IPA'

In [72]:
df_soloartesanas["Estilo"].unique()

array([' Imperial / Double IPA', ' Fruit Beer', ' Pale Ale',
       ' Weizen - Weissbier', ' White IPA', ' Kölsch',
       ' English Pale Ale', ' Lager', ' Pumpkin Ale',
       ' India Pale Ale - IPA', ' Session IPA', ' Blonde Ale',
       ' Golden Ale', ' Witbier (Belgian White)', ' Amber Ale',
       ' American Pale Ale - APA', ' Smoked Beer', ' American Amber Ale',
       ' English IPA', ' Tripel', ' Belgian Blonde Ale',
       ' Golden Braggot', ' American Golden Ale', ' Double Red IPA',
       ' American IPA', ' Saison', ' Extra Pale Ale', ' Braggot',
       ' German Ale', ' Rauchbier', ' Belgian Dubbel', ' Belgian IPA',
       ' Cream Ale', ' Strong Ale', ' Double IPA',
       ' California Common / Steam Beer', ' Farmhouse Ale',
       ' American Wheat', ' Spice / Herb / Vegetable',
       ' Pilsner - Pilsen', ' Summer Ale', ' India Pale Ale',
       ' Pilsner Ale', ' Belgian Ale', ' Biére de Garde',
       ' Wheat Ale Ecológica', ' Marzenbier', ' IPA con algarroba',
       ' Ste

In [77]:
# hay varias cervezas del mismo tipo con distinta nomenclarura
#df_soloartesanas[df_soloartesanas["Estilo"].str.contains("India Pale Ale")]["Estilo"].unique()
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("Pale Ale")]["Estilo"].unique()

array([' Pale Ale', ' English Pale Ale', ' India Pale Ale - IPA',
       ' American Pale Ale - APA', ' Extra Pale Ale', ' India Pale Ale',
       ' India Pale Ale (IPA)', ' American Pale Ale', ' America Pale Ale',
       ' Imperial Pale Ale', ' Indian Pale Ale',
       ' VIPA, Valencia Indian Pale Ale', ' IPA (Indian Pale Ale)',
       ' European Pale Ale', ' Brown Indian Pale Ale'], dtype=object)

In [91]:
# Unificamos las India Pale Ale, excepto las que tienen otros calificativos, que son Valencia y Brown:
mask_pale_ale=df_soloartesanas["Estilo"].str.contains("Pale Ale")
mask_india=df_soloartesanas["Estilo"].str.contains("India")
mask_valencia=df_soloartesanas["Estilo"].str.contains("Valencia")
mask_brown=df_soloartesanas["Estilo"].str.contains("Brown")

# Asignamos ' India Pale Ale'
df_soloartesanas.loc[(mask_pale_ale & mask_india &~ mask_valencia &~ mask_brown ),["Estilo"]]=' India Pale Ale'

In [94]:
# También hay que unificar las tipo "American Pale Ale - APA"
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("America")]["Estilo"].value_counts()

 American Pale Ale - APA    11
 American Pale Ale           8
 American IPA                7
 American Amber Ale          6
 American Brown Ale          4
 America Pale Ale            1
 American Golden Ale         1
 Summer American Wheat       1
 American Wheat              1
Name: Estilo, dtype: int64

In [97]:
# máscaras
mask_america=df_soloartesanas["Estilo"].str.contains("America")

df_soloartesanas.loc[(mask_pale_ale & mask_america ),["Estilo"]]=' America Pale Ale'

In [98]:
# cervezas Stout están bien diferenciadas
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("Stout")]["Estilo"].value_counts()

 Imperial Stout                             10
 Stout                                       8
 Russian Imperial Stout                      4
 Oatmeal Stout                               3
 Extra Stout                                 2
 Sweet Stout                                 2
 Dry Stout                                   2
 Smoke Imperial Stout                        1
 Russian Imperial Stout & Imperial Stout     1
Name: Estilo, dtype: int64

In [99]:
# las Witbier necesitan unificarse
df_soloartesanas[df_soloartesanas["Estilo"].str.contains("Wit")]["Estilo"].value_counts()

 Witbier (Belgian White)    7
 Witbier                    2
 Wit                        1
Name: Estilo, dtype: int64

In [102]:
df_soloartesanas.loc[df_soloartesanas["Estilo"].str.contains("Wit"),["Estilo"]]=' Witbier'

## La fermentación
Las cervezas se clasifican en cuanto a su fermentación como Ale (alta), Lager (baja) y espontánea, aunque en esta base de datos encontramos que las cervezas vienen clasificadas en tres tipos: Ale, Lager y Mixta.
Se ha detectado que hay 31 cervezas sin clasificar,pero que pueden ser identificadas por la información sobre su estilo. La mayoría de las cervezas tienen fermentación alta y entre ellas se encuentran las IPA, weizen, wit, stout y porter. Sin embargo, las de tipo rauchbier tienen fermentación baja.

In [103]:
df_soloartesanas["Fermentación"].unique()

array(['Ale', 'Lager', nan, 'Mixta'], dtype=object)

In [104]:
print("Sin dato:",df_soloartesanas["Fermentación"].isna().sum())
df_soloartesanas["Fermentación"].value_counts()

Sin dato: 31


Ale      376
Lager     19
Mixta      1
Name: Fermentación, dtype: int64

In [113]:
# Para las que no tienen dato de fermentación ¿cuál es su estilo?
#mask=df_soloartesanas["Fermentación"].isna()
df_soloartesanas.loc[(df_soloartesanas["Fermentación"].isna()),"Estilo"].value_counts()

 IPA                                        2
 Pale Ale                                   2
 Stout                                      2
 Amber Ale                                  1
 Kölsch                                     1
 America Pale Ale                           1
 Russian Imperial Stout & Imperial Stout    1
 Porter                                     1
 Witbier                                    1
 Double IPA                                 1
 Abbey Dubbel                               1
 Tropical IPA                               1
 Summer Ale                                 1
 Ale                                        1
 American Amber Ale                         1
 American Brown Ale                         1
 Belgian Golden Strong Ale                  1
 Sweet Stout                                1
 Imperial Pale Ale                          1
 American IPA                               1
 Weizen                                     1
 Rauchbier                        

In [122]:
# Las cervezas que tienen estilo "Ale", "IPA","Weizen","Wit","Stout" y "Porter" son de fermentación Ale
# Las tipo Rauchbier son asigno Lager

# Como solamente hay 1 Rauchbier, actualizo esa con "Lager" 
df_soloartesanas.loc[(df_soloartesanas["Estilo"].str.contains("Rauchbier") & df_soloartesanas["Fermentación"].isna()),["Fermentación"]]='Lager'

# y asigno a las demás "Ale"
df_soloartesanas.loc[(df_soloartesanas["Fermentación"].isna()),["Fermentación"]]='Ale'

# compruebo
print("Sin dato:",df_soloartesanas["Fermentación"].isna().sum())
df_soloartesanas["Fermentación"].value_counts()

Sin dato: 0


Ale      406
Lager     20
Mixta      1
Name: Fermentación, dtype: int64

In [124]:
df_soloartesanas["IBU"].unique()

array([ 60. ,  38. , 999. ,  50. ,  20. ,  28. ,  12. ,  10. ,  35. ,
        26. ,  30. ,  18. ,  70. ,  21. ,  40. ,  24. ,  13. ,  44. ,
        23. ,  31. ,  45. ,  85. ,  65. ,  52. , 100. ,  25. ,  48. ,
        22. ,  42. ,  14. ,  47. ,  75. ,  27. ,  66. ,  49. , 130. ,
        13.8, 110. ,  37. ,  46. ,  17.9,  55. ,  32. ,  15. ,  39. ,
        54. ,   8. ,  19. ,  11. ,  80. ,  56. ,  29. , 120. ,  59. ,
        23.5,  36. ,  17. ,  34. ,  22.4,  41. ,  43. ,  57.7,  84. ,
        62. ,  33. ,  90. ,  15.2,  72. ,  69. ,  64. ,  92. ,   9.9,
       150. ])

In [125]:
df_soloartesanas["Lúpulos"].unique()

array([nan, 'Citra', 'Chinook, citra, mosaic, mandarina bavaria y melón',
       'Hull melon', 'Citra y Cascade', 'Simcoe, Centennial y Galaxy',
       'Chinook, Columbus, El Dorado, Mosaic y Citra',
       'Chinook, Citra y Amarillo',
       'Cascade, Magnum, Perle, Chinook, Amarillo, Summit',
       'Columbus, Magnum, Cascade, Chinook',
       'Herkules, Perle, Cascade, Summit', 'Simcoe, Chinook, Centenial',
       'Saaz',
       'Amarillo, Cascade, Centennial, Chinook, Citra, Equinox, Simcoe',
       'Citra y Cascade (junto a dryhopping con Citra)',
       'Magnum, Chinook, Cascade y Tomahawk', 'Cascade', 'Simcoe',
       'Hellertauer Herbrucker, Fuggles', 'Amarillo, Cascade, Simcoe',
       'Simcoe, Amarillo', 'Summit, Citra y Simcoe', 'Americanos',
       'Amarillo, Cascade y Columbus',
       'Cascade, East Kent Holdings y Hallertauer Mittlefruh',
       'Chinook, Willamette y Cascade', 'Perle, Chinook, Amarillo, Citra',
       'Nugget, Cascade y Saaz', 'Centennial, Chinook, Mosa

In [126]:
df_soloartesanas["European Brewing Convention (EBC)"].unique()

array([nan, '8,1', '5,5', '15,5', '8', '4', '7', '20', '22', '12', '18',
       '15', '42', '10', '40', '13', '6,8', '5', '36', '14', '24', '2',
       '6.6', '10-15', '25', '19', '16', '34', '6', '23', '8.7', '9.9',
       '10,6', '105', '35', '21', '30', '27', '33', '26', '32', '134',
       '80', '110', '57', '100', '59', '40-45', '120', '65', '85', '32.4',
       '38.6', '38', '28', '9.3'], dtype=object)

In [127]:
df_soloartesanas["Maltas"].unique()

array([nan, 'Pale, Vienna, malta de trigo y Cara-pils',
       'Pils, Pale, Crystal y Avena', 'Extra Pale Malt',
       'Pale Ale, Pilsen', 'Pale Ale, Caramünich tipo I',
       'Pale Ale, Cara 20, Munich',
       'Pilsen, Munich, Caramunich y malta clara de trigo', 'Pale',
       'Pilsner, Cara Blond y Munich', 'Pale y Melaoidin',
       'Pilsen, Munich I, Cara-Pils, Carabelga, Viena, Chocolate, Abadia Monasteriol',
       'Maris Otter, Vienna, Trigo', 'Caramel Pils', 'Pilsen y CaraGold',
       'Pale Ale', 'Pale Ale y Crystal',
       'Pale Ale , Caramel y Melanoidin',
       'Pale, Caraamber, Munich, Trigo y Carafoam',
       'Pilsen, Munich, Carahell', 'Pale Ale, Cara Red',
       'Pale, Wheat, Munich', 'Pilsner, Caramelo',
       'Maris Otter y Crystal', 'Trigo, pilsner, carapils, pale ale',
       'Pilsen y Carahell', 'Pale, Trigo, Crystal', 'Pilsen',
       'Pale, Crystal, Pilss y Carapills', 'Cascade',
       'Pale, Wheat malta y carapils', 'Pale Ale, Crystal y Black malt.',
  

In [71]:
df_soloartesanas["Temperatura de consumo"].unique()

array([nan, '6-8 ºC', '12 ºC', 'Entre 2 y 18ºC', '6 ºC', 'Ale',
       '6 - 8º C', 'Lager', '7º - 9ºC', '6-8ºC', '4-6ºC', '3-8ºC',
       '10-13ºC', '6ºC', '4ºC', '5-12ºC', '5-8ºC', '8-12ºC', '6-7ºC',
       '8-10ºC', '7-10ºC', '6-12ºC', '6-10ºC', '7-9ºC', '2-4ºC', '3-5ºC',
       '2-6ºC', 'A partir de 9ºC', '3-5 ºC.', '4-8ºC', '7ºC', '4-7ºC',
       '4-7 ºC', '8-12 ºC', '4-10 ºC', '6-10 ºC', '4-6 ºC', '8-10 ºC',
       '10-13 ºC', '4,5-7 ºC', '8-13 ºC', '9-13 ºC', '7-10 ºC', '11 ºC',
       '14-16 ºC', 'Entre 7 y 10ºC', '10 ºC', '7 ºC', '6º - 10º C',
       '6-11ºC', '12-16ºC', '5ºC', '10-14 ºC', '8-14 ºC', '12-14ºC',
       '5º-8ºC', '+12ºC', '5-7ºC', '10ºC', '4-7 ªC', '2-4 ºC', '10-12ºC'],
      dtype=object)

### Unificamos el tipo de grano
A pesar de que este dato falta en bastantes registros, lo dejamos porque puede ser interesante para pintar. Para ello, primero unificamos los datos ya que algunos están escritos de varias formas.

In [128]:
df_soloartesanas["Grano"].unique()

array([nan, 'Cebada', 'Trigo', 'Avena', 'Arroz', 'Cebada y trigo',
       'Trigo y cebada (50%)', 'Cebada y Trigo', 'Cebada y avena'],
      dtype=object)

In [129]:
df_soloartesanas["Grano"].value_counts()

Cebada                  254
Trigo                    45
Avena                     4
Trigo y cebada (50%)      2
Cebada y Trigo            2
Cebada y avena            1
Cebada y trigo            1
Arroz                     1
Name: Grano, dtype: int64

In [130]:
# Unifico cebada y trigo
granos=(df_soloartesanas["Grano"]=='Cebada y trigo') | (df_soloartesanas["Grano"]=='Trigo y cebada (50%)') | (df_soloartesanas["Grano"]=='Cebada y Trigo')
df_soloartesanas.loc[granos,"Grano"]='Cebada y trigo'


In [131]:
df_soloartesanas.to_csv("data/soloartesanas_limpio.csv", sep = ';', index = False)

In [132]:
df_soloartesanas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 427 entries, 1 to 434
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 427 non-null    int64  
 1   color                              427 non-null    object 
 2   name                               427 non-null    object 
 3   brand                              427 non-null    object 
 4   description                        427 non-null    object 
 5   image                              427 non-null    object 
 6   Estilo                             427 non-null    object 
 7   Graduación                         427 non-null    float64
 8   Fermentación                       427 non-null    object 
 9   IBU                                427 non-null    float64
 10  Lúpulos                            98 non-null     object 
 11  European Brewing Convention (EBC)  101 non-null    object 